# Generate input for positive class

In [1]:
import os
from tqdm import tqdm
from lxml import etree
import json

# 1. Create the lexicon from terms embedded in ENE from Perdido XML-TEI

\<rs type="place">\<term type="place">

In [2]:
def get_term_occurrences_from_ene(filename):
    words = []
    if os.path.exists(filename):
        try:
            tree = etree.parse(filename)
            for term in tree.xpath('.//rs[@type="place" and @subtype="ene"]/term[@type="place"]'):
                phrase = ''
                tokens = term.xpath('.//w[@pos="N" or @pos="PREPDET" or @pos="PREP" or @pos="DET"]')
                for i, w in enumerate(tokens):
                    if len(w.text) > 1:
                        if ('DET' not in w.get('pos') and 'PREP' not in w.get('pos')):
                            phrase += w.text.lower() + ' '
                        if ('DET' in w.get('pos') or 'PREP' in w.get('pos')) and (i > 0 and i < len(tokens)-1):
                            phrase += w.text.lower() + ' '
                words.append(phrase.strip())
        except:
            pass
    return words

## 1.1 Corpus Traitement Text

In [15]:
path = '/Users/lmoncla/Documents/Data/Corpus/Choucas/Perdido'
output_filename = 'Traitement_Texte_pivot_lexicon'

words = []
for doc in tqdm(sorted(os.listdir(path))):
    filename = os.path.join(path, doc, doc+'.xml') # version Traitements_Texte
    words.extend(get_term_occurrences_from_ene(filename))

# list to dict with frequency
frequency_dict_geo_TT = {value: words.count(value) for value in words}
print('Size of the lexicon', len(frequency_dict_geo_TT))

#save the dict in file
with open(output_filename + '.json', 'w') as fp:
    json.dump(frequency_dict_geo_TT, fp, ensure_ascii=False)

100%|██████████| 95/95 [00:00<00:00, 367.59it/s]

Size of the lexicon 190


## 1.2 Corpus Visorando

In [16]:
path = '/Users/lmoncla/Documents/Data/Corpus/Visorando/Perdido'
output_filename = 'Visorando_pivot_lexicon'

words = []
for doc in tqdm(sorted(os.listdir(path))):
    filename = os.path.join(path, doc[:-4]+'.xml') # version visorando
    words.extend(get_term_occurrences_from_ene(filename))

# list to dict with frequency
frequency_dict_geo_viso = {value: words.count(value) for value in words}
print('Size of the lexicon', len(frequency_dict_geo_viso))

#save the dict in file
with open(output_filename + '.json', 'w') as fp:
    json.dump(frequency_dict_geo_viso, fp, ensure_ascii=False)

100%|██████████| 1530/1530 [00:01<00:00, 919.36it/s]


Size of the lexicon 813


## 1.3 Lexicons preview

In [6]:
for k, v in frequency_dict_geo_viso.items():
    print(k, v)

refuge 198
hameau 347
pont 209
cascade 47
lac du plan 2
lac 285
hôtel 25
bourgs 1
chalets 84
passage du lac 1
passage 46
pointe 132
refuge du col 4
torrent 56
place 228
source 30
col 769
rocher 33
dôme 3
plaine de 1
plaine 44
crête des lances 1
crête 41
rue 779
fontaine 44
usine 10
chemin 872
sentier 246
village 464
hameaux 16
chapelle 136
garage 3
téléphérique 3
gare d' arrivée 2
gare 34
glacier 21
dômes 5
église 87
camping 20
commune 51
bois 139
intersection du moulin 1
intersection 43
domaine 19
bourg 42
ruisseau 106
lieu 45
combe 71
plan 74
chalet 126
montagne 39
faubourg 4
grotte 34
forêt 86
col de 2
vallée 156
champs 19
carrefour 197
versant 21
ville 23
vallon 78
ruines de l' alpe 1
ruines 59
hameau du plan 1
massif 36
glaciers 4
piste 98
état de fraîcheur 1
état 4
 64
bâtisse de la prise 1
bâtisse 3
passage au dessus du chemin 1
maisons 36
vallées 4
route du col 23
route 592
ferme 71
carrefour de la route 3
parking de la plage 6
parking 419
ports des sables 1
ports 1
cabane 105


In [ ]:
# load the dict from file
# only necessery if you want to run the nex section and not the previous one (adapt output_filename)
with open(output_filename + '.json') as fp:
    frequency_dict_geo_viso = json.load(fp)

# 2. Find occurrences of the lexicon in the corpus (outside ENE)

In [25]:
def get_ngrams_wt_term_outside_ene(filename, frequency_dict_geo, ngram_id):
    json_content = []
    if os.path.exists(filename):
        try:
            
            tree = etree.parse(filename)
            tokens = tree.xpath('.//w')
            for i, token in enumerate(tokens):
                if token.text in frequency_dict_geo:
                    line = {'num':ngram_id, 'class':'1', 'id_phrase':'0','pivot':token.text,'occurrence': '0', 'url':os.path.join(doc, doc+'.xml')}
                    phrase = []
                    for j in range(3,0,-1):
                        try:
                            words = {'word':tokens[i-j].text, 'POS':tokens[i-j].get('pos'), 'lemma':tokens[i-j].get('lemma')}
                        except IndexError:
                            words = {'word':'_', 'POS':'_', 'lemma':'_'}
                        phrase.append(words)
                    phrase.append({'word':token.text, 'POS':token.get('pos') + '+LS', 'lemma':token.get('lemma')})
                    for j in range(1,4):
                        try:
                            words = {'word':tokens[i+j].text, 'POS':tokens[i+j].get('pos'), 'lemma':tokens[i+j].get('lemma')}
                        except IndexError:
                            words = {'word':'_', 'POS':'_', 'lemma':'_'}
                        phrase.append(words)
                    line['phrase'] = phrase
                    try:
                        print(tokens[i-3].text, tokens[i-2].text , tokens[i-1].text , '[', token.text, ']', tokens[i+1].text, tokens[i+2].text, tokens[i+3].text)
                    except IndexError:
                        pass
                    ngram_id += 1
                    json_content.append(line)
        except :
            pass
    
    return json_content

## 2.1 Corpus Traitement text

In [26]:
path = '/Users/lmoncla/Documents/Data/Corpus/Choucas/Perdido'

json_content = []
ngram_id = 1
for doc in sorted(os.listdir(path)):
    filename = os.path.join(path, doc, doc+'.xml') # version Traitements_Texte
    json_content.extend(get_ngrams_wt_term_outside_ene(filename, frequency_dict_geo_TT, ngram_id))

print('nnumber of ngram',len(json_content))

name = 'Traitement_Texte_class1'
with open(name + ".json", "w") as outfile:
    json.dump(json_content,outfile, ensure_ascii=False)

l' ascension du [ massif ] en suivant les
allemandes jusqu' au [ sommet ] où se trouve
nécropole . Le [ chemin ] retour , descend
Se rendre au [ lieu-dit ] Hirzenstein . Pour
, traverser le [ village ] de Wattwiller en
la sortie du [ village ] , à un
, à un [ carrefour ] en Y ,
garer sur le [ parking ] avant le complexe
Hirzenstein . Le [ chemin ] de départ se
d' information des [ chemins ] du club vosgien
débute par un [ chemin ] forestier situé à
talweg et son [ ruisseau ] . Non loin
jusqu' à un [ carrefour ] en T :
à droite du [ chemin ] forestier , vous
la ligne de [ crête ] ( triangle jaune
à un petit [ col ] et un éperon
partir de ce [ rocher ] fortifié , vous
vue sur la [ plaine ] d' Alsace du
monter vers le [ sommet ] en suivant la
talweg , le [ sentier ] quitte la ligne
la ligne de [ crête ] et passe d'
passe d' un [ versant ] à l' autre
talweg . Le [ sentier ] est plat jusqu'
jusqu' à un [ carrefour ] en T. Prendre
Vous arrivez au [ sommet ] de la ligne
la ligne de [ crête ] 

## 2.2 Corpus Visorando

In [27]:
path = '/Users/lmoncla/Documents/Data/Corpus/Visorando/Perdido'
json_content = []
ngram_id = 1
for doc in sorted(os.listdir(path)):
    filename = os.path.join(path, doc[:-4]+'.xml') # version visorando
    json_content.extend(get_ngrams_wt_term_outside_ene(filename, frequency_dict_geo_viso, ngram_id))
                            
print('nnumber of ngram',len(json_content))

name = 'Visorando_class1'
with open(name + ".json", "w") as outfile:
    json.dump(json_content,outfile, ensure_ascii=False)

dans la belle [ vallée ] encaissée de Champagny-le-Haut
passer devant l' [ église ] et au carrefour
église et au [ carrefour ] prendre à droite
Arrivé près du [ torrent ] , le longer
longer jusqu' au [ hameau ] de Friburge .
. Traverser le [ pont ] , puis à
prendre le petit [ sentier ] à gauche .
Suivre ainsi le [ torrent ] jusqu' au pont
torrent jusqu' au [ pont ] du Laisonnay d'
Passer devant le [ parking ] jusqu' au prochain
jusqu' au prochain [ pont ] qui enjambe la
qui enjambe la [ cascade ] de Py et
à gauche le [ sentier ] abrupt qui grimpe
le temps la [ cascade ] puis continuer après
continuer après le [ refuge ] de Plaisance ,
, franchir la [ cascade ] et contourner le
Poursuivre sur ce [ sentier ] qui passe à
à proximité du [ lac ] du Plan Séry
vue sur le [ lac ] de La Plagne
descendre vers le [ refuge ] du lac mais
le refuge du [ lac ] mais continuer par
droite . Le [ chemin ] reste sur le
vue sur le [ lac ] de la Plagne
Plagne . Le [ sentier ] remonte alors vers
alors vers u

## Get the list of terms in ENE not categorized by Perdido

In [ ]:
words = []
for doc in tqdm(sorted(os.listdir(path))):
    filename = os.path.join(path, doc, doc+'.xml') 
    if os.path.exists(filename):
        tree = etree.parse(filename)

        for term in tree.xpath('.//rs[@type="unknown" and @subtype="ene"]/term[@type="unknown"]'):
            phrase = ''
            for w in term.xpath('.//w[@pos="N" or @pos="PREPDET" or @pos="PREP"]'):
                phrase += w.text.lower() + ' '
                #print(w.text, end=' ')
            words.append(phrase.strip())
            #print()

# list to dict with frequency
frequency_dict_unknown = {value: words.count(value) for value in words}
print('Size of the lexicon', len(frequency_dict_unknown))

In [ ]:
frequency_dict_unknown

## List of files with rs in term

In [ ]:
path = '/Users/lmoncla/Documents/Data/Corpus/Visorando/Perdido'

In [ ]:
words = []
for doc in tqdm(sorted(os.listdir(path))):
    filename = os.path.join(path, doc+'.xml') 
    if os.path.exists(filename):
        tree = etree.parse(filename)


        for term in tree.xpath('.//term//rs'):
            print(filename)
            phrase = ''
            for w in term.xpath('.//w'):
                phrase += w.text.lower() + ' '
        #print(phrase, end=' ')
        #print()
            
            #print()

        # list to dict with frequency
        